In [10]:
from datetime import datetime, timedelta
import requests
import config as cfg
import ast
import psycopg


class Attempt:
    def __init__(self, user_id, oauth_consumer_key, lis_result_sourcedid, lis_outcome_service_url, is_correct, attempt_type, created_at):
        self.user_id = user_id
        self.is_correct = is_correct
        self.attempt_type = attempt_type
        self.created_at = created_at
        self.oauth_consumer_key = oauth_consumer_key
        self.lis_result_sourcedid = lis_result_sourcedid
        self.lis_outcome_service_url = lis_outcome_service_url


In [ ]:
def get_response():
    end_date = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
    start_date = end_date - timedelta(days=1)

    api_url = cfg.api_url
    q_params = {
        'client': cfg.client,
        'client_key': cfg.client_key,
        'start': start_date,
        'end': end_date
    }
    response = requests.get(api_url, params=q_params)
    return response.json()

def parse_passback_params(passback_params):
    required_keys = ['oauth_consumer_key', 'lis_result_sourcedid', 'lis_outcome_service_url']
    result = ast.literal_eval(passback_params)
    if all(key in result for key in required_keys):
        return result
    return False

def validate_attempt(attempt):
    required_keys = ['lti_user_id', 'is_correct', 'attempt_type', 'created_at']
    if not all(key in attempt for key in required_keys):
        return False
    if not isinstance(attempt.get('lti_user_id'), str):
        return False 
    
    attempt_type = attempt.get('attempt_type')
    is_correct = attempt.get('is_correct')

    if attempt_type == 'run':
        if is_correct is not None:
            return False
    elif attempt_type == 'submit':
        if is_correct != 1 and is_correct != 0:
            return False
    return True
    
def create_attempts_objects(attempt, passback_params):
    user_id = attempt.get('lti_user_id')
    oauth_consumer_key = passback_params.get('oauth_consumer_key')
    lis_result_sourcedid = passback_params.get('lis_result_sourcedid')
    lis_outcome_service_url = passback_params.get('lis_outcome_service_url')
    is_correct = bool(attempt.get('is_correct'))
    attempt_type = attempt.get('attempt_type')
    created_at = attempt.get('created_at')
    attempt_o = Attempt(user_id, oauth_consumer_key, lis_result_sourcedid, lis_outcome_service_url, is_correct, attempt_type, created_at)
    return attempt_o

def insert_data(conn, attempt_o):
    insert_query = """
    INSERT INTO user_attempts (
        user_id,
        oauth_consumer_key,
        lis_result_sourcedid,
        lis_outcome_service_url,
        is_correct,
        attempt_type,
        created_at
    )
    VALUES (
        %(user_id)s,
        %(oauth_consumer_key)s,
        %(lis_result_sourcedid)s,
        %(lis_outcome_service_url)s,
        %(is_correct)s,
        %(attempt_type)s,
        %(created_at)s
    )
    """

    with conn.cursor() as cur:
        cur.execute(insert_query, vars(attempt_o))
        

def main():
    attempts_data = get_response()
    with psycopg.connect (**cfg.db_config) as conn:
        for attempt in attempts_data:
            if not validate_attempt(attempt):
                continue
            passback_params = parse_passback_params(attempt.get('passback_params'))
            if not passback_params:
                continue
            if attempt['attempt_type'] == 'submit':
                print(attempt['is_correct'])
            attempt_o = create_attempts_objects(attempt, passback_params)
            insert_data(conn, attempt_o)

main()

0
1
0
1
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
1
1
0
0
0
1
1
1
1
1
1
1
1
1
1
0
1
0
0
1
0
0
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
0
1
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
1
1
0
1
0
0
0
1
1
0
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
1
0
0
0
1
0
1
1
0
1
0
1
1
1
1
1
1
0
1
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
1
0
0
0
0
0
0
0
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
1
1
1
0
1
1
0
0
1
1
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
1
1
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
0
0
0
0
0
0
0
0
0
1
1
1
1
1
0
0
1
1
0
0
1
0
0
1
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0


In [3]:
print(cfg.db_config)

{'dbname': 'final_project', 'user': 'bubbleraspberry', 'password': 'Morp_67890', 'host': 'localhost', 'port': 5432}
